### Limpieza general 
normalizar fondo y eliminar archivos no finalizados en .png

In [ ]:
import os
import io
from PIL import Image
from tqdm import tqdm
import shutil
import pandas as pd

version="1.21.1"
tam=32


item_folder = f"datos/item_{version}_{str(tam)}x{str(tam)}"

BACKGROUND_COLOR = (255, 255, 255)   

###  Definición de Funciones

In [ ]:
def safe_open_image(path):
    try:
        im = Image.open(path)
        im.load()
        return im
    except Exception as e:
        print(f"ERROR abriendo {path}: {e}")
        return None

def ensure_rgb_with_bg(im, background_color=BACKGROUND_COLOR):
    """
    Convierte P/RGBA a RGB con fondo.
    Usa el snippet que nos diste.
    """
    if im.mode == "P":
        im = im.convert("RGBA")

    if im.mode == "RGBA":
        bg = Image.new("RGB", im.size, background_color)
        bg.paste(im, mask=im.split()[-1])  # usa el alpha como máscara
        im = bg
    else:
        im = im.convert("RGB")
    return im

def dataset_from_folder(folder_path: str):

    image_files = []
    # Listar archivos, ignorando posibles errores si no existe la carpeta
    if os.path.exists(folder_path):
        for f in os.listdir(folder_path):
            image_files.append(f)
    else:
        print(f"Advertencia: La carpeta {folder_path} no existe.")
        return pd.DataFrame()

    images = []
    labels = []

    for filename in sorted(image_files):
        image_path = os.path.join(folder_path, filename)

        # abrir imagen 
        im = safe_open_image(image_path)
        if im is None:
            continue
            
        im = ensure_rgb_with_bg(im)

        buf = io.BytesIO()
        im.save(buf, format="PNG")

        images.append({"bytes": buf.getvalue()})

        # --- generar label desde el nombre ---
        label = os.path.splitext(filename)[0].replace("_", " ")
        labels.append(label)

    # --- dataframe final ---
    df = pd.DataFrame({
        "image": images,
        "label": labels
    })

    return df

###  Items

In [ ]:
# eliminamos el archivo compass 14 de items pq da problemas
compass_path = os.path.join(item_folder, "compass_14.png")

os.remove(compass_path)
print(f"Eliminado: {compass_path}")

# Cargar dataset de items
df_items = dataset_from_folder(folder_path=item_folder)
df_items.head()

In [ ]:
output_path = f"datos/faithful_{version}_{str(tam)}x{str(tam)}.parquet"
df_tems.to_parquet(output_path)
print(f"Dataset guardado en {output_path} con {len(df_items)} imágenes.")

### Bloques 

In [ ]:
block_folder = f"datos/block_{version}_{str(tam)}x{str(tam)}"


# Archivos que no terminan en .png 
no_png = [f for f in os.listdir(block_folder) if not f.lower().endswith(".png")]
no_png += ["debug", "debug2"] # bloques erraticos conocidos
block_to_drop = [os.path.splitext(i)[0] for i in no_png]

# eliminar en el folder TODOS los archivos cuyo basename esté en block_to_drop
for f in list(os.listdir(block_folder)):
    base = os.path.splitext(f)[0]
    if base in block_to_drop:
        fp = os.path.join(block_folder, f)
        os.remove(fp)
                   
# Cargar dataset de bloques
df_blocks = dataset_from_folder(folder_path=block_folder)
df_blocks.head()

###  Guardar 

In [ ]:
df_all = pd.concat([df_blocks, df_items], ignore_index=True)
output_path = f"datos/ib_faithful_{version}_{str(tam)}x{str(tam)}.parquet"
df_all.to_parquet(output_path)
print(f"Dataset guardado en {output_path} con {len(df_all)} imágenes.")